##### Copyright 2025 Google LLC.

安装 Agent Development Kit (ADK)<br>
配置你的API密钥以使用国内大语言模型<br>
构建你的第一个简单智能体<br>
运行你的智能体，观察它使用工具（如Google搜索）来回答问题<br>

## 安装
安装依赖
```
pip install google-adk
```

配置模型API<br>
复制.env.example文件，重命名为.env<br>

In [1]:
import os
import dotenv

dotenv.load_dotenv()
print("✅ 模型API配置成功")

✅ 模型API配置成功


导入 ADK 组件

In [2]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini  # 使用Gemini模型
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search  # 使用谷歌搜索
from google.genai import types

print("✅ ADK组件导入成功")

✅ ADK组件导入成功


构建 AI 智能体

LLM语言模型：<br>
`提示词 -> LLM -> 文本`<br>
AI 智能体：<br>
`提示词 -> 智能体 -> 思考 -> 行动 -> 观察 -> 最终答案`<br>

定义智能体

name和description：一个简单的名称和描述，用于标识我们的智能体。<br>
model：将为智能体的推理提供动力的特定 LLM。我们将使用 "gemini-2.5-flash-lite"。<br>
instruction：智能体的指导性提示词。这告诉智能体它的目标是什么以及应该如何表现。<br>
tools：智能体可以使用的[工具](https://google.github.io/adk-docs/tools/)列表。首先，我们将给它 `google_search` 工具，这让它能够在线查找最新信息。

In [3]:
# 创建搜索工具
import warnings
import datetime
warnings.filterwarnings("ignore")


def google_search(query):
    """
    使用 Google Custom Search API
    
    args:
        query: 搜索关键词
    returns:
        list: 包含搜索结果的列表，每个结果是一个字典，包含标题、URL和描述
    """
    import requests
    
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'key': os.getenv("SEARCH_API_KEY"),
        'cx': os.getenv("SEARCH_ENGINE_ID"),
        'q': query,
        'num': 3  # 返回结果数量
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        results = []
        for item in data.get('items', []):
            results.append({
                'title': item.get('title'),
                'url': item.get('link'),
                'description': item.get('snippet')
            })
        
        return results
    
    except Exception as e:
        print(f"API搜索出错: {e}")
        return []


def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    current_time = datetime.datetime.now().strftime("%Y年%m月%d日 %H:%M:%S")
    return {"current_time": current_time}


In [7]:
from google.adk.models.lite_llm import LiteLlm
# 使用LiteLlm接口适配其他厂商模型

root_agent = Agent(
    name="helpful_assistant",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use the tools google_search and get_current_time to answer user questions.",
    tools=[google_search, get_current_time],
)
print("✅ 智能体已创建")

✅ 智能体已创建


运行智能体

Runner：管理对话，将我们的消息发送给智能体，并处理其响应。<br>
创建一个 `InMemoryRunner` 并配置 `root_agent`：<br>

In [8]:
runner = InMemoryRunner(agent=root_agent)
print("✅ Runner 创建成功")

✅ Runner 创建成功


调用 `.run_debug()` 方法来发送提示词并获取答案

In [10]:
response = await runner.run_debug(
    "今天电影院正在上映哪些新电影？"
)


 ### Continue session: debug_session_id

User > 今天电影院正在上映哪些新电影？
helpful_assistant > 我来帮您查找今天电影院正在上映的新电影信息。不过，要获取最准确的信息，我需要知道您所在的城市，因为不同城市的电影排片可能不同。

让我先搜索一下今天的电影上映信息：
helpful_assistant > 让我搜索更具体的今日上映信息：
helpful_assistant > 根据搜索结果，我为您整理了以下信息：

## 今天电影院正在上映的电影信息

### 1. **从搜索结果看，可能正在热映的电影包括：**

**香港地区（根据hkmovie6.com）：**
- 《尋秦記》
- 《魔法壞女巫：第二章》
- 《阿凡達3》
- 《優獸大都會2》
- 《狂蟒之災》
- 《萬王之王》
- 《海綿寶寶：深海尋「寶」》
- 《內幕》
- 《逆轉上半場》
- 《TOMORROW X TOGETHER VR CONCERT : HEART》

**台湾地区（根据国宾影城）：**
- 《大濛（燒餅場）》
- 《海綿寶寶電影》
- 《A Dance with Rainbows》（2026/01/16上映）
- 《Dust Bunny》

### 2. **近期/今日可能上映的新电影：**
- 《一个完整的陌生人》（A Complete Unknown）- 2024年12月25日在美国上映
- 其他圣诞档期电影

### 3. **重要提醒：**

由于电影上映信息具有**极强的地域性和时效性**，最准确的方法是：

1. **告诉我您所在的城市** - 我可以帮您查询当地时间
2. **使用本地电影票务APP**：
   - **中国大陆**：猫眼电影、淘票票、豆瓣电影
   - **香港**：hkmovie6.com、各大院线官网
   - **台湾**：国宾影城、威秀影城官网
   - **其他地区**：当地主流票务平台

3. **今日上映特点**：
   - 12月通常是圣诞档期，会有多部新片上映
   - 动画电影、合家欢电影较多
   - 部分电影可能只在特定地区上映

**建议您：**
- 打开您常用的电影票务APP
- 选择您所在的城市
- 查看"正在热映"或"今日上映"栏目

您愿意告诉我您所

智能体如何工作的？

智能体执行了搜索以获取关于ADK的最新信息，它知道使用这个工具是因为：

1. 智能体会检查并了解它可以使用哪些工具。
2. 智能体的指令指定了使用搜索工具来获取最新信息，或者在不确定答案时使用。

查看智能体思考和行动的完整、详细追踪的最佳方式是在**ADK web UI**中，我们将在本notebook的后面部分设置它。

我们将在课程后面介绍更详细的日志记录和可观测性工作流程。

2.5 轮到你了！

这是你观察智能体实际运行的机会。向它提出一个需要最新信息的问题。

试试下面这些问题，或者自己想一个：

- 伦敦的天气怎么样？
- 谁赢得了上一届足球世界杯？
- 现在电影院正在上映哪些新电影？

In [11]:
response = await runner.run_debug("阿凡达3的豆瓣评分是多少？")


 ### Continue session: debug_session_id

User > 阿凡达3的豆瓣评分是多少？
helpful_assistant > 我来帮您查询《阿凡达3》的豆瓣评分。
helpful_assistant > 根据搜索结果，我为您找到了《阿凡达3》的豆瓣评分信息：

## 《阿凡达3：火与烬》豆瓣评分

### **评分：7.6分**

### **详细评分信息：**
- **评分人数**：169,137人评价
- **电影全名**：《阿凡达：火与烬》（Avatar: The Seed Bearer）
- **其他名称**：阿凡达3 / 阿凡达3：火与烬 / 阿凡达3：火与灰 / 阿凡达3：带种者
- **IMDb编号**：tt1757678

### **评分特点：**
1. **系列最低分**：根据搜索结果，7.6分是《阿凡达》系列电影中**史上最低的评分**
2. **评价人数众多**：已有超过16.9万人参与评分
3. **近期上映**：电影于2025年12月上映，评分还在持续更新中

### **电影简介：**
影片聚焦杰克·萨利与奈蒂莉一家的命运起伏，在前作的情感余波之上，深刻描绘一个家族在战火中如何成长、并共同守护血脉相连的情感纽带。

### **评分对比参考：**
- 《阿凡达》（2009年）：豆瓣评分8.8分
- 《阿凡达：水之道》（2022年）：豆瓣评分8.0分
- 《阿凡达：火与烬》（2025年）：豆瓣评分7.6分

可以看出，《阿凡达3》的评分确实比前两部有所下降，但7.6分仍然是一个不错的分数，说明电影整体质量还是受到观众认可的，只是可能没有达到前作的高度。

您对这部电影感兴趣吗？需要我帮您查询更多相关信息吗？



第3节：尝试 ADK Web 界面

ADK 包含一个内置的 Web 界面，用于与你的智能体进行交互式聊天、测试和调试。

<img width="1200" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/adk-web-ui.gif" alt="ADK Web UI" />

要使用 ADK web UI，你需要使用 `adk create` 命令创建一个包含 Python 文件的智能体。

运行下面的命令来生成一个 `sample-agent` 文件夹，其中包含所有必要的文件，包括用于你的代码的 `agent.py`、预配置了 API 密钥的 `.env` 文件，以及一个 `__init__.py` 文件：

In [ ]:
!adk create sample_agent --model LiteLlm(model="deepseek/deepseek-chat") --api_key $DEEPSEEK_API_KEY


Agent created in d:\code\agent_learn\agent-course-google2025\Day1\sample-agent:
- .env
- __init__.py
- agent.py



Now we can run ADK web:

In [10]:
!adk web

^C


现在你可以使用上面的链接访问 ADK 开发 UI。

In [ ]:
# +-----------------------------------------------------------------------------+
# | ADK Web Server started                                                      |
# |                                                                             |
# | For local testing, access at http://127.0.0.1:8000.                         |
# +-----------------------------------------------------------------------------+